In [1]:
using Revise
using Bilevel

using RigidBodyDynamics

┌ Info: Precompiling Bilevel [9688c538-179f-11e9-3174-495cea6b7f67]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /Users/blandry/.julia/compiled/v0.7/ForwardDiff/k0ETY.ji for ForwardDiff [f6369f11-7733-5829-9624-2563aa707210]
└ @ Base loading.jl:1185
┌ Info: Recompiling stale cache file /Users/blandry/.julia/compiled/v0.7/Ipopt/yMQMo.ji for Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1185
┌ Info: Recompiling stale cache file /Users/blandry/.julia/compiled/v0.7/RigidBodyDynamics/WeevQ.ji for RigidBodyDynamics [366cf18f-59d5-5db9-a4de-86a9f6786172]
└ @ Base loading.jl:1185
┌ Warning: Deprecated syntax `@doc call with ->` at /Users/blandry/.julia/packages/RigidBodyDynamics/uyInU/src/spatial/frame.jl:34.
│ Use `a line break` instead.
└ @ ~/.julia/packages/RigidBodyDynamics/uyInU/src/spatial/frame.jl:34
┌ Warning: Deprecated syntax `@doc call with ->` at /Users/blandry/.julia/packages/RigidBodyDynamics/uyInU/src/spatial/frame.jl:45.
│ Use `a line break` 

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), [0.,0.,0.])
normal = FreeVector3D(default_frame(floor), [0.,0.,1.])
floor_obs = Obstacle(floor, point, normal, :xyz, .5)

obstacles = [floor_obs]

1-element Array{Obstacle,1}:
 Obstacle(RigidBody: "floor", Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false)

In [3]:
env = Environment(mechanism, urdf, obstacles)

Environment(Contact[Contact(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., ball, Point3D in "after_floor_to_ball": [0.0, 0.0, 0.0], Obstacle(floor, Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false))])

In [5]:
sim_data = get_sim_data_indirect(mechanism,env,.01,relax_comp=false)

SimData(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., StateCache{…}(…), StateCache{…}(…), Bilevel.EnvironmentCache(Bilevel.ContactJacobianCache[ContactJacobianCache{Dual}(Contact(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., ball, Point3D in "after_floor_to_ball": [0.0, 0.0, 0.0], Obstacle(floor, Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false)), [0.0 9.81 … -9.81 -1.80207e-15; 0.0 0.0 … 1.20138e-15 -9.81; 9.81 0.0 … 0.0 0.0], Dual{Nothing}(0.0), ForwardDiff.Dual[Dual{Nothing}(1.0) Dual{Nothing}(0.0) Dual{Nothing}(0.0); Dual{Nothing}(0.0) Dual{Nothing}(1.0) Dual{Nothing}(0.0); Dual{Nothing}(0.0) Dual{Nothing}(0.0) Dual{Nothing}(1.0)], ForwardDiff.Dual[Dual{Nothing}(

Compute the resulting force as a free vector at the contact point (D \beta + c_n)

Rotate this free vector in the world frame

Transform the contact point from the body (with the contact point) frame to the root frame

linear is just the force

angular is point \times force

Compute the resulting joint torque from the wrench using the geometric jacobian

In [26]:
ec = Bilevel.EnvironmentCache(env, sim_data.xn_cache[Float64])

Bilevel.EnvironmentCache(Bilevel.ContactJacobianCache[ContactJacobianCache{Float64}(Contact(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., ball, Point3D in "after_floor_to_ball": [0.0, 0.0, 0.0], Obstacle(floor, Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false)), [0.0 9.81 … -9.81 -1.80207e-15; 0.0 0.0 … 1.20138e-15 -9.81; 9.81 0.0 … 0.0 0.0], 0.0, [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [0.0, 0.0, 0.0], GeometricJacobian: body: "world", base: "after_floor_to_ball", expressed in "world":
[-1.0 -0.0 -0.0 -0.0 -0.0 -0.0; -0.0 -1.0 -0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -1.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -1.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0 -1.0 -0.0; -0.0 -0.0 -0.0 -0.0 -0.0 -1.0], nothing, [0.0 -0.0 0.0; 0.0 0.0 -0.0; -0.0 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 

In [53]:
set_configuration!(sim_data.xn_cache[Float64], [1.,0.,0.,0.,1.,2.,0.])
Bilevel.contact_jacobian!(ec, sim_data.xn_cache[Float64])
ec.contact_jacobians[1].J * [1.,0.,1.,2.,1.]

UndefVarError: UndefVarError: contact_trans not defined